In [1]:
import cv2
from PIL import Image
import argparse
from pathlib import Path
from multiprocessing import Process, Pipe,Value,Array
import torch
from config import get_config
from mtcnn import MTCNN
from Learner import face_learner
from utils import load_facebank, draw_box_name, prepare_facebank
import matplotlib.pyplot as plt

In [2]:
conf = get_config(False)

mtcnn = MTCNN()
print('mtcnn loaded')

learner = face_learner(conf, True)
learner.threshold = conf.threshold
if conf.device.type == 'cpu':
    learner.load_state(conf, 'cpu_final.pth', True, True)
else:
    learner.load_state(conf, 'final.pth', True, True)
learner.model.eval()
print('learner loaded')

mtcnn loaded
{'data_path': WindowsPath('data'), 'work_path': WindowsPath('work_space'), 'model_path': WindowsPath('work_space/models'), 'log_path': WindowsPath('work_space/log'), 'save_path': WindowsPath('work_space/save'), 'input_size': [112, 112], 'embedding_size': 512, 'use_mobilfacenet': False, 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': WindowsPath('data/faces_vgg_112x112'), 'ms1m_folder': WindowsPath('data/faces_ms1m_112x112'), 'emore_folder': WindowsPath('data/faces_emore'), 'batch_size': 100, 'facebank_path': WindowsPath('data/facebank'), 'threshold': 1.5, 'face_limit': 10, 'min_face_size': 50}
ir_se_50 model generated
learner loaded


In [3]:
update = False   # whether perform update the facebank
tta = True      # whether test time augmentation

if update:
    targets, names = prepare_facebank(conf, learner.model, mtcnn, tta = tta)
    print('facebank updated')
else:
    targets, names = load_facebank(conf)
    print('facebank loaded')

facebank loaded


In [16]:
frame = cv2.imread(r"data\facebank\P00973\2023-11-16_06-58-28_SYePjI4Iq0.jpg")

# image = Image.fromarray(frame[...,::-1]) #bgr to rgb
image = Image.fromarray(frame)
bboxes, faces = mtcnn.align_multi(image, conf.face_limit, conf.min_face_size)
bboxes = bboxes[:,:-1] #shape:[10,4],only keep 10 highest possibiity faces
bboxes = bboxes.astype(int)
bboxes = bboxes + [-1,-1,1,1] # personal choice    
results, score = learner.infer(conf, faces, targets, tta)
for idx,bbox in enumerate(bboxes):
    frame = draw_box_name(bbox, names[results[idx] + 1] + '_{:.2f}'.format(score[idx]), frame)
cv2.imshow("f", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
frame = cv2.imread(r"data\facebank\P00033\2023-07-08_21-01-21_4Ku3fd5tX4.jpg")
image = Image.fromarray(frame)
bboxes, faces = mtcnn.detect_faces(image, min_face_size=50)
print(bboxes)

[[155.89641237 242.81780243 259.6726802  376.72889587   0.99372113]]
